___

<a href='https://github.com/eliasmelul/'> <img src='https://s3.us-east-2.amazonaws.com/wordontheamazon.com/NoMargin_NewLogo.png' style='width: 15em;' align='right' /></a>
# Finance with Python
### Capital Asset Pricing Model
___
<h4 align="right">by Elias Melul, Data Scientist </h4> 

___


The CAPM model describes the relationship between expected returns and volatility (systematic risk). Why does this matter? Because investors expect to be compensated for risk and time value of money. So the CAPM is used as a theoretical model that adjusts for risk when evaluating the value of a stock.

This model assumes the existance of a market portfolio - all possible investments in the world combined - hence the existance of a risk-free asset. However, this is not true. It also assumes that all investors are rational, and therefore hold the optimal portfolio. This is in consequence of the mutual fund theorem: _all investors hold the same portfolio of risky assets, the tangency portfolio_. Therefore, the CAPM assumes that the tangency portfolio is the market portfolio. Again... not true.

In this context, the tangency portfolio is the portfolio with the largest Sharpe Ratio. But what is the _Sharpe Ratio?_


**Sharpe Ratio**: measures the performance of a security compared to a risk-free asset, after adjusting for its risk. This is the excess return per unit of risk of an investment.
$$
Sharpe = \frac{\overline{r_{i}} - r_f}{\sigma_{i}}
$$
        When Sharpe > 1, GOOD risk-adjusted returns
    
        When Sharpe > 2, VERY GOOD risk-adjusted returns
    
        When Sharpe > 3, EXCELLENT risk-adjusted returns


_How do we measure risk?_ There are many ways to measure risk, although variance (standard deviation) is one of the most common. However, when it comes to the risk that cannot be avoided through diversification, the Beta is king!

**Beta**: measures the market risk that cannot be avoided through diversification. This is the relationship between the stock and the market portfolio. In other words, it is a measure of how much risk the investment will add to a portfolio that looks like the market.
$$ 
\beta_{i} = \frac{\sigma_{i,m}}{\sigma_{m}^2}
$$

        When beta = 0, it means that there's no relationship.
    
        When beta < 1, it means that the stock is defensive (less prone to high highs and low lows)
    
        When beta > 1, it means that the stock is aggresive (more prone to high highs and low lows)
        
Amazing! We're only one small step away. The risk-adjusted returns. 

**Expected Return CAPM**: calculates the expected return of a security adjusted to the risk taken. This equates to the return expected from taking the extra risk of purchasing this security.
$$
\overline{r_{i}} = r_f + \beta_{i}(\overline{r_{m}} - r_f) 
$$

Awesome! There are a couple more things we will discuss later, but for now, now that we understand the underlying theory of the CAPM model, let's get coding!

---

**Step 1**:Import necessary libraries

In [1]:
import numpy as np
import pandas as pd
from pandas_datareader import data as wb

**Step 2**: Import data for a stock and the market data. In this case we will use:
    1. Amazon
    2. S&P 500 (as a proxy for the market)

In [2]:
from pandas_datareader import data as pdr
import yfinance as yfin
tickers = ['AMZN','^GSPC']
data = pd.DataFrame()

yfin.pdr_override()
for t in tickers:
    data[t] = pdr.get_data_yahoo(t, start='2015-1-1')["Adj Close"]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


**Step 3**: compute the logarthmic returns of the daily data. This is logarithmic daily returns of the data. 
    
    Why logarithmic and not simple returns?
    
        We usually use logarithmic returns when making calculations about a single asset over time.
        We use simple returns when dealing with multiple assets over the same timeframe.

In [3]:
sec_returns = np.log(data / data.shift(1))
sec_returns.head()

,AMZN,^GSPC
Date,,
2015-01-02,NaN,NaN
2015-01-05,-0.020731,-0.018447
2015-01-06,-0.023098,-0.008933
2015-01-07,0.010544,0.011563
2015-01-08,0.006813,0.017730


**Step 4**: Compute covariance and market variance.

    As we can see from the Beta function in the introduction, we need the covariance between Amazon stock and the market. We also need the variance of market returns. We need these annualized, so we will multiply them by 252, since there are 252 trading days in an actual year. 

In [4]:
cov = sec_returns.cov() *252
cov

,AMZN,^GSPC
AMZN,0.111165,0.039291
^GSPC,0.039291,0.035209


In [5]:
cov_with_market = cov.iloc[0,1]
cov_with_market

0.03929086869949053

In [6]:
market_var = sec_returns['^GSPC'].var()*252
market_var

0.03520906423305716

**Step 5**: Calculate Beta
$$ 
\beta_{pg} = \frac{\sigma_{pg,m}}{\sigma_{m}^2}
$$

In [7]:
amazon_beta = cov_with_market / market_var
amazon_beta

1.115930501288388

**Step 6**: Expected Return CAPM
$$
\overline{r_{pg}} = r_f + \beta_{pg}(\overline{r_{m}} - r_f) 
$$

Here, we need to make a couple assumptions. 
1. A 10 year US government bond is a good proxy for a risk-free asset, with a yield of 2.5%
2. The common risk premium is between 4.5% and 5.5%, so we will use 5%. Risk premium is the expected return of the market minus the risk-free return.


In [8]:
riskfree = 0.025
riskpremium = 0.05
amazon_capm_return = riskfree + amazon_beta*riskpremium
amazon_capm_return

0.0807965250644194

Let's try the same calculation, but this time, we use the mean of the returns of the market as part of the risk premium.

In [9]:
riskfree = 0.025
riskpremium = (sec_returns['^GSPC'].mean()*252) - riskfree
amazon_capm_return = riskfree + amazon_beta*riskpremium
amazon_capm_return

0.09422307344733366

Using the returns of the market yields a higher risk-adjusted returns. This makes sense since the market has been doing well since 2010 (the beginning of our dataset). There's only one more important calculation left: the _Sharpe Ratio_

**Step 7: Sharpe Ratio**
$$
Sharpe = \frac{\overline{r_{amazon}} - r_f}{\sigma_{amazon}}
$$

In [10]:
log_returns = np.log(data / data.shift(1))
sharpe_amazon = (amazon_capm_return-riskfree)/(log_returns['AMZN'].std()*250**0.5)
sharpe_amazon

0.20844747470333336

##### There it goes!

The next part just creates functions for each of these metrics, so that we can easily access them for any stock with a simple function.

In [11]:
from datetime import datetime

In [12]:
#Import the data of any stock of set of stocks
def import_stock_data(tickers, start = '2010-1-1', end = datetime.today().strftime('%Y-%m-%d')):
    data = pd.DataFrame()
    if len([tickers]) ==1:
        data[tickers]  = pdr.get_data_yahoo(tickers, start = start)['Adj Close']
        data = pd.DataFrame(data)
    else:
        for t in tickers:
            data[t]  = pdr.get_data_yahoo(t, start = start)['Adj Close']
    return(data)

In [13]:
data = import_stock_data(['AMZN','^GSPC'], start = '2010-1-1')
data.head()

[*********************100%***********************]  2 of 2 completed


,AMZN,^GSPC
Date,,
2010-01-04,6.6950,1132.989990
2010-01-05,6.7345,1136.520020
2010-01-06,6.6125,1137.140015
2010-01-07,6.5000,1141.689941
2010-01-08,6.6760,1144.979980


In [14]:
def compute_beta(data, stock, market):
    log_returns = np.log(data / data.shift(1))
    cov = log_returns.cov()*250
    cov_w_market = cov.loc[stock,market]
    market_var = log_returns[market].var()*250
    return cov_w_market/market_var

In [15]:
compute_beta(data, 'AMZN',"^GSPC")

1.1098972998325392

In [16]:
def compute_capm(data, stock, market, riskfree = 0.025, riskpremium = 'market'):
    log_returns = np.log(data / data.shift(1))
    if riskpremium == 'market':
        riskpremium = (log_returns[market].mean()*252) - riskfree
    beta = compute_beta(data, stock, market)
    return (riskfree + (beta*riskpremium))

In [17]:
compute_capm(data, 'AMZN', '^GSPC')

0.10730822746647772

In [18]:
def compute_sharpe(data, stock, market, riskfree = 0.025, riskpremium='market'):
    log_returns = np.log(data / data.shift(1))
    ret = compute_capm(data, stock, market, riskfree, riskpremium)
    return ((ret-riskfree)/(log_returns[stock].std()*250**0.5))

In [19]:
compute_sharpe(data, "AMZN","^GSPC")

0.24979184939102933

In [20]:
def stock_CAPM(stock_ticker, market_ticker, start_date = '2010-1-1', riskfree = 0.025, riskpremium = 'set'):
    data = import_stock_data([stock_ticker,market_ticker], start = start_date)
    beta = compute_beta(data, stock_ticker, market_ticker)
    capm = compute_capm(data, stock_ticker, market_ticker)
    sharpe = compute_sharpe(data, stock_ticker, market_ticker)
    #listcapm = [beta,capm,sharpe]
    capmdata = pd.DataFrame([beta,capm,sharpe], columns=[stock_ticker], index=['Beta','Return','Sharpe'])
    return capmdata.T

In [21]:
stock_CAPM("AAPL","^GSPC")

[*********************100%***********************]  2 of 2 completed


,Beta,Return,Sharpe
AAPL,1.111689,0.107441,0.289768


Let's give it a go with a different stock. NVIDIA!

In [22]:
stock='NVDA'
market='^GSPC'
data = import_stock_data([stock,market], start = '2000-1-1')
print(f"Beta: {round(compute_beta(data,stock,market),4)}")
print(f"CAPM Expected Return: {round(compute_capm(data, stock, market),4)}")
print(f"Sharpe Ratio: {round(compute_sharpe(data, stock, market),4)}")

[*********************100%***********************]  2 of 2 completed
Beta: 1.6329
CAPM Expected Return: 0.0595
Sharpe Ratio: 0.0578
